# Обучение нейронной сети на базе датасета Mushrooms

## Установим все необхоидимое

In [1]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("maysee/mushrooms-classification-common-genuss-images")


print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mushrooms-classification-common-genuss-images' dataset.
Path to dataset files: /kaggle/input/mushrooms-classification-common-genuss-images


In [3]:
# указываю полный путь до папок
data_path = f'{path}/Mushrooms'
data_path

'/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms'

In [4]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Класс датасета

In [7]:
class Mushrooms(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

        self.data_list = []
        self.len_dataset = 0
        self.classes_count = {0: 0, 1: 0, 2: 0, 3: 0,
                              4: 0, 5: 0, 6: 0, 7: 0, 8: 0}
        self.broken_files = []

        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = dir_list
                self.class_to_idx = {
                    cls: idx for idx, cls in enumerate(dir_list)
                }
                continue

            cls = path_dir.split('/')[-1]

            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)

                # здесь, видимо в процессе сжатия, у меня появился поломанный
                # файл, поэтому я решил пожертвовать скоростью, лишь бы избежать
                # поломанных файлов
                try:
                    Image.open(file_path).convert('RGB')
                    self.data_list.append((file_path, self.class_to_idx[cls]))
                    self.classes_count[self.class_to_idx[cls]] += 1
                except Exception as e:
                    print(file_path)
                    self.len_dataset -= 1
                    self.broken_files.append(file_path)

            self.len_dataset += len(file_list)

    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = Image.open(file_path).convert('RGB')

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

In [8]:
# трансформация картинок
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

## Инициализация датасета и разбиение на train/validate/test

In [9]:
# инициализация датасета
data = Mushrooms(data_path, transform)

# выводим вспомогательную инфу
print('length:', len(data))
print('elements at classes: ', data.classes_count)
print('classes to indexces: ', data.class_to_idx)

/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms/Russula/092_43B354vYxm8.jpg
length: 6713
elements at classes:  {0: 364, 1: 311, 2: 316, 3: 353, 4: 750, 5: 1563, 6: 1147, 7: 1073, 8: 836}
classes to indexces:  {'Entoloma': 0, 'Suillus': 1, 'Hygrocybe': 2, 'Agaricus': 3, 'Amanita': 4, 'Lactarius': 5, 'Russula': 6, 'Boletus': 7, 'Cortinarius': 8}


Разобьем на train/validate/test

In [10]:
# инициализирую размеры частей
train_size = int(0.8 * len(data)) # 0.8
val_size = int(0.1 * len(data)) # 0.1
test_size = len(data) - train_size - val_size # 0.1

# создадим train/validate/split
train_data, val_data, test_data = random_split(data, [train_size, val_size, test_size])

print('train length:', len(train_data))
print('validate length:', len(val_data))
print('test lenght:', len(test_data))

train length: 5370
validate length: 671
test lenght: 672


Классы не сбалансированы, поэтому их надо было забалансить на батчах, чтобы нормально обучить модель

Реализацию подсмотрел [тут](https://traceback.ru/balance-classes-in-torch-with-weighted-random-sampler/). Следующие два блока кода как раз этим и занимался. Также забатчил датасет


In [11]:
from collections import Counter

train_counts = Counter(im[1] for im in train_data)
w = [1 / train_counts[im[1]] for im in train_data]

print(w)
print(train_counts)

[0.003484320557491289, 0.0007886435331230284, 0.001098901098901099, 0.00390625, 0.001098901098901099, 0.0017006802721088435, 0.001098901098901099, 0.001142857142857143, 0.001098901098901099, 0.001098901098901099, 0.001098901098901099, 0.001098901098901099, 0.0015313935681470138, 0.001142857142857143, 0.003484320557491289, 0.0007886435331230284, 0.0007886435331230284, 0.0007886435331230284, 0.0007886435331230284, 0.001098901098901099, 0.001098901098901099, 0.003484320557491289, 0.001142857142857143, 0.0035460992907801418, 0.00398406374501992, 0.001098901098901099, 0.001142857142857143, 0.0017006802721088435, 0.0007886435331230284, 0.0035460992907801418, 0.001142857142857143, 0.0017006802721088435, 0.001098901098901099, 0.0007886435331230284, 0.0007886435331230284, 0.001098901098901099, 0.0035460992907801418, 0.001098901098901099, 0.0015313935681470138, 0.00390625, 0.0035460992907801418, 0.0007886435331230284, 0.0015313935681470138, 0.003484320557491289, 0.0007886435331230284, 0.00153139

In [12]:
from torch.utils.data.sampler import WeightedRandomSampler

sampler = WeightedRandomSampler(w, num_samples=len(w))

train_loader = DataLoader(train_data, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

## Класс нейронной сети

В качестве нейронной сети выбрал модель VGG. Далее реализую ее

In [18]:
class VGG11(nn.Module):
    def __init__(self, num_classes=9, dropout=0.5): # Reduced dropout
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            nn.Conv2d(256, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            nn.Conv2d(512, 512,(3, 3), padding=1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, (3, 3), padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten()

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        out = self.classifier(x)
        return out

In [19]:
from torch.optim import Adam

model = VGG11().to(device)
loss_model = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.0001)

In [ ]:
from sklearn.metrics import f1_score

EPOCHS = 10

for epoch in range(EPOCHS):

    # Training phase
    model.train()
    running_train_loss = []
    train_preds = []
    train_targets = []
    train_loop = tqdm(train_loader, leave=False)

    for x, targets in train_loop:
        x = x.to(device)
        targets = targets.to(device)

        pred = model(x)
        loss = loss_model(pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss.append(loss.item())
        mean_train_loss = sum(running_train_loss) / len(running_train_loss)

        train_preds.extend(torch.argmax(pred, dim=1).cpu().tolist())
        train_targets.extend(targets.cpu().tolist())

        train_loop.set_description(f'Epoch[{epoch+1} / {EPOCHS}], train_loss={mean_train_loss:.4f}')

    train_f1 = f1_score(train_targets, train_preds, average='macro')

    # Validation phase
    model.eval()
    running_val_loss = []
    val_preds = []
    val_targets = []
    val_loop = tqdm(val_loader, leave=False)

    with torch.no_grad():
        for x, targets in val_loop:
            x = x.to(device)
            targets = targets.to(device)

            pred = model(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss) / len(running_val_loss)

            val_preds.extend(torch.argmax(pred, dim=1).tolist())
            val_targets.extend(targets.tolist())

            val_loop.set_description(f'Epoch[{epoch+1} / {EPOCHS}], val_loss={mean_val_loss:.4f}')

    val_f1 = f1_score(val_targets, val_preds, average='macro')


    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {mean_train_loss:.4f}, Train F1: {train_f1:.4f}, Val Loss: {mean_val_loss:.4f}, Val F1: {val_f1:.4f}")

Epoch [1/10] Train Loss: 2.1208, Train F1: 0.1249, Val Loss: 2.0460, Val F1: 0.1443


Epoch [2/10] Train Loss: 1.9872, Train F1: 0.1954, Val Loss: 1.9854, Val F1: 0.2036


Epoch [3/10] Train Loss: 1.8613, Train F1: 0.2822, Val Loss: 1.9825, Val F1: 0.2364


Epoch [4/10] Train Loss: 1.6940, Train F1: 0.3658, Val Loss: 1.9660, Val F1: 0.2594


Epoch [5/10] Train Loss: 1.4956, Train F1: 0.4520, Val Loss: 1.9182, Val F1: 0.2835


Epoch [6/10] Train Loss: 1.2421, Train F1: 0.5564, Val Loss: 1.9701, Val F1: 0.2773


Epoch [7/10] Train Loss: 0.8988, Train F1: 0.6970, Val Loss: 2.2649, Val F1: 0.3270


Epoch [8/10] Train Loss: 0.6491, Train F1: 0.7797, Val Loss: 2.3611, Val F1: 0.3042


Epoch [9/10] Train Loss: 0.4469, Train F1: 0.8523, Val Loss: 2.7653, Val F1: 0.3001


Epoch [10/10] Train Loss: 0.3186, Train F1: 0.9005, Val Loss: 2.7101, Val F1: 0.3024
